### Example notebook for model training and inference
- Author: Xiaohui Zhang

In [6]:
import tensorflow as tf
import importlib
import os, random
import json
import sys
import numpy as np
import argparse

from model_dau_nores import model
from dataloader_sleep_tfrecords_local import dataloader_sleep_tfrecords

In [33]:
# Define training configuration
args = argparse.Namespace()

# Set training config attributes
args.mode = "train"
args.data_path = '/shared/planck/MRI/xiaohui/mouse_optical/sleep_stage/2022-Ben-10s-raw-masked-tempzscore-broadband-states-tfrecords'
args.seed = 42

args.batch_size = 4
args.num_classes = 3
args.num_frames = 168
args.num_epochs = 200
args.lr_init = 0.0001
args.num_rnn_units = 64
args.loss = "focal"
args.model_name = "demo"
args.model_savedir = f"../Models/{args.model_name}"
args.hemispheric = False
args.model_continue = False
args.logs_dir = f"../Logs/{args.model_name}"

In [34]:
class project_params:
    def __init__(self, params):
        self.params = params
        
config = project_params(args)

In [35]:
# Initialize both dataloader and model 
sleep_dataloader = dataloader_sleep_tfrecords(config)
sleep_scoring_model = model(config)

Using focal loss...



In [36]:
# Training
train_dataset, val_dataset, test_dataset = sleep_dataloader.load_data()
sleep_scoring_model.train(train_dataset, val_dataset)

14472 1809 1810
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
Model: "classifier"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 168, 128, 1  0           []                               
                                28, 1)]                                                           
                                                                                                  
 time_distributed_25 (TimeDistr  (None, 168, 128, 12  640        ['input_2[0][0]']                
 ibuted)                        8, 64)                                                            
                                                                                                  
 time_distributed_26 (TimeDistr  (None, 168, 128,

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
Epoch 1/200


2024-06-04 23:32:42.043250: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2024-06-04 23:32:47.281378: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-04 23:32:47.282634: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-04 23:32:47.282665: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2024-06-04 23:32:47.283251: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-06-04 23:32:47.283340: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2024-06-04 23:32:52.054547: W tensorflow/core/common_runtime/bf

ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[672,64,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node swap_in_gradient_tape/classifier/time_distributed_28/leaky_re_lu_11/LeakyRelu/LeakyReluGrad_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[div_no_nan_1/ReadVariableOp_1/_6]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[672,64,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node swap_in_gradient_tape/classifier/time_distributed_28/l

In [ ]:
# Inference
sleep_scoring_model = model(config)
model.test(test_dataset)